<a href="https://colab.research.google.com/github/ramoants254/RNN-Project/blob/main/intro_to_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importations

In [130]:
!wget 'https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py'

--2024-08-21 09:50:15--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.3’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-08-21 09:50:15 (88.7 MB/s) - ‘helper_functions.py.3’ saved [10246/10246]



In [131]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [132]:
!wget 'https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip'

--2024-08-21 09:50:16--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.132.207, 74.125.201.207, 74.125.202.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.132.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.3’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2024-08-21 09:50:17 (84.8 MB/s) - ‘nlp_getting_started.zip.3’ saved [607343/607343]



In [133]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [134]:
unzip_data('nlp_getting_started.zip')

## Visualizing

In [135]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [136]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [137]:
df=train_df.sample(frac=1,random_state=42)
df.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [138]:
train_df['target'].value_counts()

,count
target,
0,4342
1,3271


In [139]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [140]:
train_df.shape

(7613, 5)

In [141]:
test_df.shape

(3263, 4)

In [142]:
X_train,X_test,y_train,y_test=train_test_split(df['text'].to_numpy(),
                                               df['target'].to_numpy(),
                                               test_size=0.2,
                                               random_state=42)

In [143]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((6090,), (1523,), (6090,), (1523,))

In [144]:
 X_train[:10],y_train[:10]

(array(['Detonation fashionable mountaineering electronic watch water-resistant couples leisure tab\x89Û_ http://t.co/E61x9Y65QD http://t.co/OVLET0gDqm',
        "Men escape car engulfed in flames in Parley's Canyon crews investigating cause - http://t.co/tFan6qq2Ys http://t.co/rAkwWritPo",
        'I-77 Mile Marker 31 to 40 South Mooresville  Iredell Vehicle Accident Congestion at 8/6 1:18 PM',
        "#Greece's tax revenues collapse as debt crisis continues via @guardian #bailout http://t.co/cJvbQXw83s ^mp",
        'Be not afraid of sudden fear neither of the desolation of the wicked when it cometh. For the Lord shall be thy... http://t.co/bP597YDs2b',
        'Storm batters Auckland and Northland: A violent overnight storm has battered Auckland and Northland uprooting... http://t.co/enrPGRgtTs',
        "Benzema increasingly looks to be a casualty of Benitez's new look squad. Arsenal bound? 50-50 chance I think",
        'Acquire your postexistence straight a elevation in addition

## Converting Text to Numbers

In [145]:
# # Define a TextVectorization layer
# tv = TextVectorization(max_tokens=100000,
#                        standardize='lower_and_strip_punctuation',
#                        split='whitespace',
#                        ngrams=None,
#                        output_mode='int',
#                        output_sequence_length=None,
#                         pad_to_max_tokens=True
#                        )


tv = TextVectorization(max_tokens=None,
                       standardize='lower_and_strip_punctuation',
                       split='whitespace',
                       ngrams=None,
                       output_mode='int',
                       output_sequence_length=15  # Set sequence length
                       )


In [146]:
tv.adapt(X_train)


In [147]:
# Apply the TextVectorization layer to your data
sample_vectorized_text = tv(X_train[:10])
print(sample_vectorized_text)

tf.Tensor(
[[  572  3567  3340  1968   128  2995  2831  3413  3080 13769 12581     0
      0     0     0]
 [  595  1047   111   416     4   243     4  1871  1565   638  1282   322
  12097 12331     0]
 [ 2254  1665  3373  1450     5   366   519  4664  4892   881   124 17208
     17  3826  5873]
 [ 3520  2459  3193   156    26  2315   709  1729    47  1700 18259 13931
    528     0     0]
 [   21    34  2374     6  4135   218  3328     6     2   508     6     2
   3902    45    15]
 [   81  3779  5670     7  4607     3   327  3287    81    39  5635  5670
      7  4607  6588]
 [18108 11255   352     5    21     3   671     6 18116    51   158  1477
   3799  2880 19031]
 [ 5774    33  8911  1242     3 16321     4  5766     5   516  2251   548
   4518 17802     0]
 [ 9683    46    12   152     2  4133   102   188     8  2400    21  2107
      5   603    19]
 [10798  2641   638 15660    11   703   182    94   151  5526    57     0
      0     0     0]], shape=(10, 15), dtype=int64)


## Creating an Embeddding using Embedding

In [148]:
embedding=layers.Embedding(input_dim=100000,
                           output_dim=128
                          )
embedding

<Embedding name=embedding_2, built=False>

In [149]:
embedded_sample=embedding(tv(X_train[:10]))

In [150]:
embedded_sample

<tf.Tensor: shape=(10, 15, 128), dtype=float32, numpy=
array([[[-0.03804253, -0.00560905,  0.04483138, ...,  0.0434189 ,
          0.00541832,  0.02974447],
        [ 0.04061426,  0.03990084, -0.02692847, ...,  0.01719201,
         -0.02063659, -0.02424226],
        [ 0.01344608,  0.03929085, -0.04168533, ...,  0.00111014,
         -0.01600607,  0.02125803],
        ...,
        [-0.03865596, -0.0034804 ,  0.04031647, ..., -0.04359453,
         -0.02671039, -0.00388693],
        [-0.03865596, -0.0034804 ,  0.04031647, ..., -0.04359453,
         -0.02671039, -0.00388693],
        [-0.03865596, -0.0034804 ,  0.04031647, ..., -0.04359453,
         -0.02671039, -0.00388693]],

       [[-0.02808509,  0.01311782, -0.03800507, ...,  0.04794571,
          0.04438046,  0.04755164],
        [ 0.00666964,  0.00461857,  0.00338693, ..., -0.04094741,
         -0.04260899,  0.03113076],
        [ 0.03841413,  0.02366999, -0.04042941, ..., -0.0294093 ,
         -0.02639853,  0.00968631],
        ...,

## Modelling

In [151]:
model=Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf',MultinomialNB())
])

model.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

## Evaluating the model

In [152]:
model_evaluation=model.score(X_test,y_test)
model_evaluation

0.799080761654629

In [153]:
model_predict=model.predict(X_test)
model_predict[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

## Function to Evaluate

In [154]:
def calulate_results(y_true,y_pred):
    model_accuracy=accuracy_score(y_true,y_pred)*100
    model_precision=precision_score(y_true,y_pred)*100
    model_recall=recall_score(y_true,y_pred)*100
    model_f1=f1_score(y_true,y_pred)*100
    model_results={
        'accuracy':model_accuracy,
        'precision':model_precision,
        'recall':model_recall,
        'f1':model_f1
    }
    return model_results

In [155]:
calulate_results(y_test,model_predict)

{'accuracy': 79.9080761654629,
 'precision': 88.27292110874201,
 'recall': 62.255639097744364,
 'f1': 73.01587301587301}

## Dense Model

In [156]:
SAVE_DIR='model_logs'

In [157]:
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=tv(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_1=tf.keras.Model(inputs,outputs,name='model_1')


In [158]:
model_1.summary()

Model: "model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_2                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 15, 128)             │      12,800,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_2           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,800,129 (48.83 MB)

 Trainable params: 12,800,129 (48.83 MB)

 Non-trainable params: 0 (0.00 B)

In [159]:
model_1.compile(loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])

In [160]:
model_1_history=model_1.fit(X_train,y_train,
                            epochs=5,
                            validation_data=(X_test,y_test),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                    experiment_name='model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20240821-095018
Epoch 1/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 44s 216ms/step - accuracy: 0.6537 - loss: 0.6531 - val_accuracy: 0.7557 - val_loss: 0.5444
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 82s 217ms/step - accuracy: 0.8214 - loss: 0.4580 - val_accuracy: 0.7820 - val_loss: 0.4776
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 82s 217ms/step - accuracy: 0.8799 - loss: 0.3285 - val_accuracy: 0.7892 - val_loss: 0.4643
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 42s 217ms/step - accuracy: 0.9258 - loss: 0.2336 - val_accuracy: 0.7886 - val_loss: 0.4665
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 82s 220ms/step - accuracy: 0.9424 - loss: 0.1826 - val_accuracy: 0.7938 - val_loss: 0.4843


In [161]:
model_1_results=model_1.evaluate(X_test,y_test)
model_1_results

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7871 - loss: 0.4976


[0.48433244228363037, 0.7938279509544373]

In [162]:
model_1_predict=model_1.predict(X_test)
model_1_predict[:10]

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


array([[0.38772044],
       [0.84568274],
       [0.99727637],
       [0.06810702],
       [0.13000338],
       [0.9419837 ],
       [0.96229285],
       [0.9926253 ],
       [0.93938935],
       [0.19728528]], dtype=float32)

In [163]:
y_test[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [164]:
model_1_predict_classes=tf.squeeze(tf.round(model_1_predict))
model_1_predict_classes[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [165]:
model_1_results=calulate_results(y_test,model_1_predict_classes)
model_1_results

{'accuracy': 79.3827971109652,
 'precision': 81.3953488372093,
 'recall': 68.42105263157895,
 'f1': 74.34640522875817}

In [166]:
model_1_results

{'accuracy': 79.3827971109652,
 'precision': 81.3953488372093,
 'recall': 68.42105263157895,
 'f1': 74.34640522875817}

## Recurrent Neural Network

In [167]:
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=tv(inputs)
x=embedding(x)
x=layers.LSTM(64,return_sequences=True)(x)
x=layers.LSTM(64)(x)
x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_2=tf.keras.Model(inputs,outputs,name='model_2')


In [168]:
model_2.summary()


Model: "model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_2                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 15, 128)             │      12,800,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 15, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,886,657 (49.16 MB)

 Trainable params: 12,886,657 (49.16 MB)

 Non-trainable params: 0 (0.00 B)

In [169]:
model_2.compile(loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])
model_2_history=model_2.fit(X_train,y_train,
                            epochs=5,
                            validation_data=(X_test,y_test),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                    experiment_name='model_2_rnn')])

Saving TensorBoard log files to: model_logs/model_2_rnn/20240821-095631
Epoch 1/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 51s 236ms/step - accuracy: 0.9356 - loss: 0.2356 - val_accuracy: 0.7886 - val_loss: 0.7515
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 80s 230ms/step - accuracy: 0.9741 - loss: 0.0703 - val_accuracy: 0.7702 - val_loss: 0.8162
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 83s 234ms/step - accuracy: 0.9813 - loss: 0.0512 - val_accuracy: 0.7820 - val_loss: 0.8879
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 82s 233ms/step - accuracy: 0.9821 - loss: 0.0450 - val_accuracy: 0.7669 - val_loss: 0.9283
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 83s 238ms/step - accuracy: 0.9896 - loss: 0.0293 - val_accuracy: 0.7669 - val_loss: 1.1964


In [170]:
model_2_results=model_2.evaluate(X_test,y_test)
model_2_results

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7671 - loss: 1.2233


[1.1963661909103394, 0.7669073939323425]

In [171]:
model_2_predict=model_2.predict(X_test)
model_2_predict[:10]

48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step


array([[6.6487104e-02],
       [2.7714682e-01],
       [9.9998093e-01],
       [5.9567315e-05],
       [2.2446882e-04],
       [9.9987090e-01],
       [9.9972117e-01],
       [9.9997783e-01],
       [9.9996352e-01],
       [5.9787667e-01]], dtype=float32)

In [172]:
model_2_predict_classes=tf.squeeze(tf.round(model_2_predict))
model_2_predict_classes[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [173]:
model_2_results=calulate_results(y_test,model_2_predict_classes)
model_2_results

{'accuracy': 76.69074195666448,
 'precision': 74.14330218068535,
 'recall': 71.57894736842105,
 'f1': 72.83856159143077}

## Model 3 using GRU

In [174]:
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=tv(inputs)
x=embedding(x)
x=layers.GRU(64,return_sequences=True)(x)
x=layers.GRU(64)(x)
x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_3=tf.keras.Model(inputs,outputs,name='model_3')
model_3.compile(loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])
model_3_history=model_3.fit(X_train,y_train,
                            epochs=5,
                            validation_data=(X_test,y_test),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                    experiment_name='model_3_gru')])
model_3_results=model_3.evaluate(X_test,y_test)
model_3_results

Saving TensorBoard log files to: model_logs/model_3_gru/20240821-100330
Epoch 1/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 63s 290ms/step - accuracy: 0.9465 - loss: 0.1958 - val_accuracy: 0.7702 - val_loss: 0.7520
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 47s 247ms/step - accuracy: 0.9892 - loss: 0.0290 - val_accuracy: 0.7827 - val_loss: 0.9467
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 47s 247ms/step - accuracy: 0.9935 - loss: 0.0220 - val_accuracy: 0.7781 - val_loss: 0.9585
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 47s 243ms/step - accuracy: 0.9917 - loss: 0.0264 - val_accuracy: 0.7774 - val_loss: 0.9702
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 82s 246ms/step - accuracy: 0.9928 - loss: 0.0170 - val_accuracy: 0.7774 - val_loss: 1.0012
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7734 - loss: 1.0302


[1.0012285709381104, 0.777413010597229]

In [175]:
model_3_predict=model_3.predict(X_test)
model_3_predict[:10]

48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step


array([[1.3434710e-01],
       [3.6742806e-01],
       [9.9981874e-01],
       [6.9261048e-05],
       [8.2585297e-04],
       [9.9906546e-01],
       [9.9432802e-01],
       [9.9984270e-01],
       [9.9969310e-01],
       [2.0988858e-01]], dtype=float32)

In [176]:
model_3_predict_classes=tf.squeeze(tf.round(model_3_predict))
model_3_predict_classes[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [177]:
model_3_results=calulate_results(y_test,model_3_predict_classes)
model_3_results

{'accuracy': 77.74130006565989,
 'precision': 78.397212543554,
 'recall': 67.66917293233082,
 'f1': 72.63922518159805}

## Model 4: Bidirectional

In [178]:
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=tv(inputs)
x=embedding(x)
x=layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x=layers.Bidirectional(layers.GRU(64))(x)
x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_4=tf.keras.Model(inputs,outputs,name='model_4')


In [179]:
model_4.summary()

Model: "model_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)          │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_2                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 15, 128)             │      12,800,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 15, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 128)                 │          74,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,981,633 (49.52 MB)

 Trainable params: 12,981,633 (49.52 MB)

 Non-trainable params: 0 (0.00 B)

In [180]:
model_4.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [181]:
model_4_history=model_4.fit(X_train,y_train,
                            epochs=5,
                            validation_data=(X_test,y_test),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                    experiment_name='model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240821-100854
Epoch 1/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 63s 278ms/step - accuracy: 0.9646 - loss: 0.1447 - val_accuracy: 0.7735 - val_loss: 0.6646
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 50s 264ms/step - accuracy: 0.9939 - loss: 0.0178 - val_accuracy: 0.7597 - val_loss: 0.9935
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 81s 261ms/step - accuracy: 0.9917 - loss: 0.0176 - val_accuracy: 0.7754 - val_loss: 1.2387
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 83s 265ms/step - accuracy: 0.9932 - loss: 0.0164 - val_accuracy: 0.7735 - val_loss: 1.5452
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 51s 267ms/step - accuracy: 0.9927 - loss: 0.0114 - val_accuracy: 0.7702 - val_loss: 1.2870


In [182]:
model_4_results=model_4.evaluate(X_test,y_test)
model_4_results

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7668 - loss: 1.3094


[1.28697669506073, 0.770190417766571]

In [183]:
model_4_predict=model_4.predict(X_test)
model_4_predict[:10]

48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step


array([[3.7775949e-01],
       [6.1052543e-01],
       [9.9999833e-01],
       [3.4547446e-04],
       [2.5637794e-04],
       [9.9999744e-01],
       [9.9929148e-01],
       [9.9999952e-01],
       [9.9999416e-01],
       [3.0805588e-01]], dtype=float32)

In [184]:
model_4_predict_classes=tf.squeeze(tf.round(model_4_predict))
model_4_predict_classes[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [185]:
model_4_results=calulate_results(y_test,model_4_predict_classes)
model_4_results

{'accuracy': 77.01904136572554,
 'precision': 75.36231884057972,
 'recall': 70.37593984962406,
 'f1': 72.78382581648523}

## Model 5 :CNN

In [186]:
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=tv(inputs)
x=embedding(x)
x=layers.Conv1D(filters=64,kernel_size=5,activation='relu',strides=1,padding='valid')(x)
x=layers.GlobalMaxPool1D()(x)
x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_5=tf.keras.Model(inputs,outputs,name='model_5')

In [187]:
model_5.summary()

Model: "model_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)          │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_2                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 15, 128)             │      12,800,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 11, 64)              │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_1               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,845,249 (49.00 MB)

 Trainable params: 12,845,249 (49.00 MB)

 Non-trainable params: 0 (0.00 B)

In [188]:
model_5.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [189]:
model_5_history=model_5.fit(X_train,y_train,
                            epochs=5,
                            validation_data=(X_test,y_test),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                    experiment_name='model_5_cnn')])

Saving TensorBoard log files to: model_logs/model_5_cnn/20240821-101427
Epoch 1/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 44s 219ms/step - accuracy: 0.9739 - loss: 0.1304 - val_accuracy: 0.7774 - val_loss: 1.0232
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 43s 225ms/step - accuracy: 0.9911 - loss: 0.0212 - val_accuracy: 0.7702 - val_loss: 1.0347
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 80s 216ms/step - accuracy: 0.9920 - loss: 0.0174 - val_accuracy: 0.7623 - val_loss: 1.1123
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 41s 215ms/step - accuracy: 0.9914 - loss: 0.0177 - val_accuracy: 0.7669 - val_loss: 1.0527
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 83s 221ms/step - accuracy: 0.9940 - loss: 0.0124 - val_accuracy: 0.7590 - val_loss: 1.1957


In [190]:
model_5_results=model_5.evaluate(X_test,y_test)
model_5_results

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7576 - loss: 1.2087


[1.1956764459609985, 0.7590282559394836]

In [191]:
model_5_predict=model_5.predict(X_test)
model_5_predict[:10]

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


array([[5.5381459e-01],
       [8.5968751e-01],
       [9.9996209e-01],
       [3.7193409e-04],
       [6.2586951e-05],
       [9.9946034e-01],
       [9.9909759e-01],
       [9.9999309e-01],
       [9.9994308e-01],
       [8.1281233e-01]], dtype=float32)

In [192]:
model_5_predict_classes=tf.squeeze(tf.round(model_5_predict))
model_5_predict_classes[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [193]:
model_5_results=calulate_results(y_test,model_5_predict_classes)
model_5_results

{'accuracy': 75.90282337491793,
 'precision': 72.43975903614458,
 'recall': 72.33082706766916,
 'f1': 72.38525206922498}

## Model 6: Transfer learning

In [194]:
ten_percent_df=df[['text','target']].sample(frac=0.1,random_state=42)

In [195]:
ten_percent_df.head()

,text,target
4955,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0
584,FedEx no longer to transport bioterror germs i...,0
7411,Gunmen kill four in El Salvador bus attack: Su...,1
5950,@camilacabello97 Internally and externally scr...,1
5541,Radiation emergency #preparedness starts with ...,1


In [196]:
ten_percent_text=ten_percent_df['text'].to_list()
ten_percent_labels=ten_percent_df['target'].to_list()

In [197]:
len(ten_percent_text),len(ten_percent_labels)

(761, 761)

In [198]:
all_model_results=pd.DataFrame({
    'model':model_1_results,
    'model_2':model_2_results,
    'model_3':model_3_results,
    'model_4':model_4_results,
    'model_5':model_5_results

})



In [199]:
all_model_results

,model,model_2,model_3,model_4,model_5
accuracy,79.382797,76.690742,77.741300,77.019041,75.902823
precision,81.395349,74.143302,78.397213,75.362319,72.439759
recall,68.421053,71.578947,67.669173,70.375940,72.330827
f1,74.346405,72.838562,72.639225,72.783826,72.385252


In [200]:
model_5.save('model_5.keras')

In [201]:
loaded_model=tf.keras.models.load_model('model_5.keras',
                                        custom_objects={'KerasLayer':hub.KerasLayer})

In [202]:
loaded_model.summary()

Model: "model_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)          │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_2                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 15, 128)             │      12,800,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 11, 64)              │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_1               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,535,749 (147.00 MB)

 Trainable params: 12,845,249 (49.00 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 25,690,500 (98.00 MB)

In [203]:
loaded_model.evaluate(X_test,y_test)

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7576 - loss: 1.2087


[1.1956764459609985, 0.7590282559394836]

## Getting the Most wrong Example

In [204]:
val_df=pd.DataFrame({"test":X_test,
                     "target":y_test,
                     "pred":model_5_predict_classes,
                     "pred_prob":model_5_predict.flatten()})

In [205]:
val_df.head()

,test,target,pred,pred_prob
0,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0,1.0,0.553815
1,FedEx no longer to transport bioterror germs i...,0,1.0,0.859688
2,Gunmen kill four in El Salvador bus attack: Su...,1,1.0,0.999962
3,@camilacabello97 Internally and externally scr...,1,0.0,0.000372
4,Radiation emergency #preparedness starts with ...,1,0.0,0.000063


In [206]:
most_wrong=val_df[val_df['target']!=val_df['pred']].sort_values('pred_prob',ascending=False)

In [207]:
most_wrong.head()

,test,target,pred,pred_prob
698,åÈMGN-AFRICAå¨ pin:263789F4 åÈ Correction: Ten...,0,1.0,0.999999
846,Mourning notices for stabbing arson victims st...,0,1.0,0.999998
1476,December 2011 court dismissed the group charge...,0,1.0,0.999998
881,@adorableappple No reported flooding po in the...,0,1.0,0.999998
1282,Conning Builds Strong Case for Portfolio #Dive...,0,1.0,0.999997
